In [1]:
import pandas as pd
import numpy as np

In [2]:
raw = pd.read_csv("./raw_stars.csv")
raw

,id,hip,hd,hr,gl,bf,proper,ra,dec,dist,...,bayer,flam,con,comp,comp_primary,base,lum,var,var_min,var_max
0,0,NaN,NaN,NaN,NaN,NaN,Sol,0.000000,0.000000,0.0000,...,NaN,NaN,NaN,1,0,NaN,1.000000,NaN,NaN,NaN
1,1,1.0,224700.0,NaN,NaN,NaN,NaN,0.000060,1.089009,219.7802,...,NaN,NaN,Psc,1,1,NaN,9.638290,NaN,NaN,NaN
2,2,2.0,224690.0,NaN,NaN,NaN,NaN,0.000283,-19.498840,47.9616,...,NaN,NaN,Cet,1,2,NaN,0.392283,NaN,NaN,NaN
3,3,3.0,224699.0,NaN,NaN,NaN,NaN,0.000335,38.859279,442.4779,...,NaN,NaN,And,1,3,NaN,386.901132,NaN,NaN,NaN
4,4,4.0,224707.0,NaN,NaN,NaN,NaN,0.000569,-51.893546,134.2282,...,NaN,NaN,Phe,1,4,NaN,9.366989,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119609,119611,NaN,NaN,NaN,NN 4381,NaN,NaN,23.963895,38.629391,16.9492,...,NaN,NaN,NaN,1,119611,NaN,0.002200,NaN,NaN,NaN
119610,119612,NaN,NaN,NaN,NN 4385,NaN,NaN,23.996567,47.762093,16.7224,...,NaN,NaN,NaN,1,119612,NaN,0.000088,NaN,NaN,NaN
119611,119613,NaN,NaN,NaN,NN 4386,NaN,NaN,23.996218,-44.067905,18.5185,...,NaN,NaN,NaN,1,119613,NaN,0.002224,NaN,NaN,NaN
119612,119614,NaN,NaN,NaN,NN 4387,NaN,NaN,23.997386,-34.111986,12.8205,...,NaN,NaN,NaN,1,119614,NaN,0.001086,NaN,NaN,NaN


In [129]:
named = raw.rename(columns={"proper": "name"})
named["name"].fillna(named["bf"].str.replace(" +", " ", regex=True), inplace=True)
named = named[["name", "x", "y", "z", "spect"]]
named.dropna(inplace=True)
named

,name,x,y,z,spect
0,Sol,0.000005,0.000000,0.000000,G2V
88,Tau Phe,119.736878,0.562356,-136.823418,G8III
122,The Oct,14.902031,0.103716,-64.888859,K2III
145,29 Psc,139.077212,1.106982,-7.355907,B7III-IV
154,30 Psc,131.716394,1.126662,-13.877167,M3III
...,...,...,...,...,...
119133,36 Oph,-1.036233,-5.250282,-2.680239,K1 Ve
119193,69Tau Oph,0.690292,-51.492963,-7.404135,dF3 J
119257,Gam CrA,4.078969,-13.679989,-10.782412,F8 V
119431,78Mu 2Cyg,16.290512,-10.974411,10.773027,G2 V


In [219]:
nearby = named.copy()
nearby["dist"] = (nearby.x**2 + nearby.y**2 + nearby.z**2)**0.5
nearby.sort_values(["dist"], inplace=True)
nearby[["x", "y", "z"]] = nearby[["x", "y", "z"]].round(2)

dist = 100

nearby = nearby[(nearby["x"].abs() < dist) & (nearby["y"].abs() < dist) & (nearby["z"].abs() < 200)]
nearby.drop(columns="dist", inplace=True)
nearby

,name,x,y,z,spect
0,Sol,0.00,0.00,0.00,G2V
70666,Proxima Centauri,-0.47,-0.36,-1.15,M5Ve
71453,Alp2Cen,-0.50,-0.41,-1.16,K1V
71456,Rigil Kentaurus,-0.50,-0.41,-1.16,G2V
87665,Barnard's Star,-0.02,-1.82,0.15,sdM4
...,...,...,...,...,...
81460,Eta1TrA,-25.23,-70.64,-188.46,B7IV
22818,Eta Men,14.86,51.14,-197.88,K6III
105691,8Bet Cep,55.22,-42.89,198.11,B2IIIv SB
29417,36 Cam,-5.05,90.04,199.90,K2II-III


In [247]:
typed = nearby.copy()
sizes = "0|I|II|III|IV|V|V|VI|VII"
typed["size"] = typed["spect"].replace(r"[^0IV]*([0IV]+).*", r"\1", regex=True).replace(r"0(.+)", "\1", regex=True).replace(r"[^0IV]+", np.nan, regex=True)
typed["spect"] = typed["spect"].replace(r".*([OBAFGKMLT][0-9]).*", r"\1", regex=True)
typed.dropna(inplace=True)
typed.drop_duplicates(subset="name", inplace=True)
typed["id"] = np.arange(len(typed))
typed.reset_index(inplace=True)
typed = typed[["id", "name", "x", "y", "z", "spect", "size"]]
typed

,id,name,x,y,z,spect,size
0,0,Sol,0.00,0.00,0.00,G2,V
1,1,Proxima Centauri,-0.47,-0.36,-1.15,M5,V
2,2,Alp2Cen,-0.50,-0.41,-1.16,K1,V
3,3,Rigil Kentaurus,-0.50,-0.41,-1.16,G2,V
4,4,Lalande 21185,-2.00,0.50,1.50,M2,V
...,...,...,...,...,...,...,...
1572,1572,Eta1TrA,-25.23,-70.64,-188.46,B7,IV
1573,1573,Eta Men,14.86,51.14,-197.88,K6,III
1574,1574,8Bet Cep,55.22,-42.89,198.11,B2,III
1575,1575,36 Cam,-5.05,90.04,199.90,K2,II


In [263]:
typed.iloc[41:44]

,id,name,x,y,z,spect,size
41,41,5Bet Vir,-10.91,0.44,0.34,F8,V
42,42,Denebola,-10.63,0.51,2.77,A3,V
43,43,13The Per,5.48,4.77,8.43,F7,V


In [275]:
separated = typed[:500].copy()

def dist(star1, star2):
    return ((star1.x - star2.x)**2 + (star1.y - star2.y)**2)**0.5

def add(val, plus):
    return np.round(val + plus if val >= 0 else val - plus, 2)

def nearest(vals, val):
    result = 10e10
    item = None

    for i, v in vals.iterrows():
        if v.id == val.id:
            continue

        d = dist(v, val)

        if d < result and (val.x**2 + val.y**2) >= (v.x**2 + v.y**2):
            item = v
            result = d

    return result, item


i = 1
n = len(separated)
MIN_DIST = 0.3

while True:
    if i == n:
        break

    print(i)

    min_dist, nearest_star = nearest(separated, separated.iloc[i])

    if min_dist > MIN_DIST:
        i += 1
        continue

    separated.loc[i, "x"] = add(separated.loc[i, "x"], MIN_DIST - min_dist + 0.05)
    separated.loc[i, "y"] = add(separated.loc[i, "y"], MIN_DIST - min_dist + 0.05)
    i = max(1, i - 10)

separated

1
2
1
2
3
1
2
1
2
3
4
5
6
7
8
9
10
11
12
13
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
31
32
33
34
35
36
37
38
39
40
41
42
43
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
102
103
104
105
106
107
108
109
110
111
112
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
153
154
155
156
157
158
159
160
161
162
163
164
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168


,id,name,x,y,z,spect,size
0,0,Sol,0.00,0.00,0.00,G2,V
1,1,Proxima Centauri,-0.47,-0.36,-1.15,M5,V
2,2,Alp2Cen,-1.12,-1.03,-1.16,K1,V
3,3,Rigil Kentaurus,-0.79,-0.70,-1.16,G2,V
4,4,Lalande 21185,-2.00,0.50,1.50,M2,V
...,...,...,...,...,...,...,...
495,495,78The2Tau,17.21,40.87,12.61,A7,III
496,496,4 Ser,-30.30,-34.80,0.30,A4,V
497,497,32 Per,21.42,25.86,31.68,A3,V
498,498,59 Leo,-44.57,11.79,4.93,A5,III


In [276]:
abs = separated.copy()
abs[["x", "y", "z"]] -= abs[["x", "y", "z"]].min()
abs

,id,name,x,y,z,spect,size
0,0,Sol,44.57,42.33,45.25,G2,V
1,1,Proxima Centauri,44.10,41.97,44.10,M5,V
2,2,Alp2Cen,43.45,41.30,44.09,K1,V
3,3,Rigil Kentaurus,43.78,41.63,44.09,G2,V
4,4,Lalande 21185,42.57,42.83,46.75,M2,V
...,...,...,...,...,...,...,...
495,495,78The2Tau,61.78,83.20,57.86,A7,III
496,496,4 Ser,14.27,7.53,45.55,A4,V
497,497,32 Per,65.99,68.19,76.93,A3,V
498,498,59 Leo,0.00,54.12,50.18,A5,III


In [277]:
abs.to_json("stars.json", "records")